In [1]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

please note that this notebook is very rough, i am using it as a messy workspace for exploring spotipy and developing foundational functions that I will use more later. 

# WHAT AM I ACTUALLY DOING
i want to figure out some sort of system or interface that allows navigation through the user's playlists or listening history and gives a visualized breakdown of their content. from there i would like to make tools or preset playlist generation methods using some user-selected set of songs to generate a playlist or queue through some novel mean. this could incorporate NLP techniques to group or suggest songs by lyrical content or other language-based info.

currently i have infrastructure to gather data from a user's own playlists and/or history, my immediate next steps are:
* begin learning Flask (or something similar) and make an interface for navigating a user's listening history and selecting a set of "input" songs
* after Flask is understood and set up, figure out how to allow other user's to sign in and connect their spotify activity (currently this is all just running on my personal Spotify developer account, and only accesses my own personal data)
* explore spotipys built-in song suggestion functions and see how they could be useful

In [2]:
# Replace with your own credentials from the Spotify Developer Dashboard
client_id = "c5029ed3aca649949ef00903e6aabf91"
client_secret = "836208c86df64c4d98db67b6bc426ace"

# Initialize the SpotifyOAuth object with the required scopes and redirect URI
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri="http://localhost:8000/callback",
        scope="user-read-recently-played,playlist-read-private,user-modify-playback-state",  # Add additional scopes as needed
    )
)


In [15]:
# parking lot for finished functions :)

def get_songlist_from_playlist(playlist:str) -> list:
    playlist = sp.playlist(playlist)
    songlist = []
    for i in range(len(playlist['tracks']['items'])):
        songlist.append(playlist['tracks']['items'][i]['track']['uri'])
    return songlist

def get_features_from_song(ID:str) -> dict:
    info = sp.track(ID)
    features = sp.audio_features(ID)[0]
    songdata = {'name':info['name'],
                   'uri':info['uri'],
                   'first_artist':info['artists'][0]['name'], #will i ever need to process features or collab type songs? revisit this perhaps
                   'album':info['album']['name'],
                   'duration_s':(info['duration_ms']/1000),
                   'release_date':info['album']['release_date'],   
                   'danceability':features['danceability'],
                   'energy':features['energy'],
                   'loudness':features['loudness'],
                   'speechiness':features['speechiness'],
                   'instrumentalness':features['instrumentalness'],
                   'acousticness':features['acousticness'],
                   'liveness':features['liveness'],
                   'tempo':features['tempo'],
                   'mode':features['mode'],
                   'key':features['key'],
                   'time_signature':features['time_signature'],
                  }
    return songdata

def make_df_from_songlist(songlist:list):
    df = []
    for ID in songlist:
        df.append(get_features_from_song(ID))
    return pd.DataFrame(df)

In [16]:
playlists = sp.current_user_playlists()
names = [i['name'] for i in playlists['items']]
uris = [i['uri'] for i in playlists['items']]
user_playlists = [[names[i], uris[i]] for i in range(len(names))]
user_playlists
test = get_songlist_from_playlist(user_playlists[13][1])

In [17]:
make_df_from_songlist(test)

,name,uri,first_artist,album,duration_s,release_date,danceability,energy,loudness,speechiness,instrumentalness,acousticness,liveness,tempo,mode,key,time_signature
0,Reelin' In The Years,spotify:track:1x1XQqhBViz4opcpwc7FVs,Steely Dan,Can't Buy A Thrill,277.467,1972-11-01,0.494,0.540,-17.340,0.0348,0.000000,0.15400,0.0878,134.976,1,2,4
1,Southern Nights,spotify:track:7kv7zBjMtVf0eIJle2VZxn,Glen Campbell,Southern Nights,180.026,1977-01-01,0.697,0.813,-8.890,0.0315,0.023200,0.09890,0.5830,95.301,0,11,4
2,Brown Eyed Girl,spotify:track:3yrSvpt2l1xhsV9Em88Pul,Van Morrison,Blowin' Your Mind!,183.306,1967-09,0.491,0.583,-10.964,0.0376,0.000000,0.18500,0.4060,150.566,1,7,4
3,Me and Julio Down by the Schoolyard,spotify:track:6vxHp3CDNo0afgKGp2yi1E,Paul Simon,Paul Simon,164.813,1972-01-14,0.626,0.816,-10.593,0.1310,0.000002,0.23700,0.1040,105.307,1,9,4
4,Blister In The Sun,spotify:track:7jIAttgQTpLDoNtykIQXjH,Violent Femmes,Violent Femmes,145.706,1983-01-01,0.726,0.537,-8.896,0.1140,0.000000,0.31600,0.0707,96.889,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Come and Get Your Love - Single Version,spotify:track:7GVUmCP00eSsqc4tzj1sDD,Redbone,Come and Get Your Love,205.933,1973-11-01,0.731,0.804,-7.621,0.0473,0.000280,0.17200,0.0685,107.383,0,11,4
96,Hey Jude - Remastered 2015,spotify:track:0aym2LBJBk9DAYuHHutrIl,The Beatles,1 (Remastered),425.653,2000-11-13,0.386,0.607,-7.700,0.0261,0.000014,0.01120,0.0880,147.207,1,10,4
97,More Than a Feeling,spotify:track:1QEEqeFIZktqIpPI4jSVSF,Boston,Boston,285.133,1976,0.377,0.681,-8.039,0.0298,0.002300,0.00088,0.0504,108.789,1,7,4
98,Sunflower - Spider-Man: Into the Spider-Verse,spotify:track:3KkXRkHbMCARz0aVfEt68P,Post Malone,Spider-Man: Into the Spider-Verse (Soundtrack ...,158.040,2018-12-14,0.760,0.479,-5.574,0.0466,0.000000,0.55600,0.0703,89.911,1,2,4


In [41]:
search = input("what song: ")
results = sp.search(search, limit=1)
print(get_features_from_song(results['tracks']['items'][0]['uri']))

what song:  tubthumping


{'name': 'Tubthumping', 'uri': 'spotify:track:22HYEJveCvykVDHDiEEmjZ', 'first_artist': 'Chumbawamba', 'album': 'Tubthumper', 'duration_s': 278.778, 'release_date': '1997-09-01', 'danceability': 0.348, 'energy': 0.88, 'loudness': -7.366, 'speechiness': 0.0978, 'instrumentalness': 0, 'acousticness': 0.128, 'liveness': 0.357, 'tempo': 101.185, 'mode': 1, 'key': 2, 'time_signature': 4}


In [4]:
playlists = sp.current_user_playlists()
names = [i['name'] for i in playlists['items']]
uris = [i['uri'] for i in playlists['items']]
user_playlists = [[names[i], uris[i]] for i in range(len(names))]
user_playlists

[['daylist', 'spotify:playlist:37i9dQZF1EP6YuccBxUcC1'],
 ['My Playlist #56', 'spotify:playlist:2Kh6ekhTxFTqU7qxNoGCL7'],
 ['ROAD TRIP', 'spotify:playlist:78XQXbqjOVCm18gHGbhZDY'],
 ['favs rn', 'spotify:playlist:125XcIkEadQiPK2zzb00gM'],
 ['Collie Creek', 'spotify:playlist:6WAt4gtVVVPlteBBjkoLXs'],
 ['Functioning', 'spotify:playlist:7wWMn2LHI2GBPmtT7pN3nb'],
 ['TF22', 'spotify:playlist:6oCAtyExYgtXRZvZ4CM1aj'],
 ['Gone Birding 📋', 'spotify:playlist:3W8RkensDIpi73p76U5PNh'],
 ['battery acid', 'spotify:playlist:2wVODnzg3156JuejXzwrj7'],
 ['every night is emo night', 'spotify:playlist:7dpRdMQ2JYbQxAMsBcwZuH'],
 ['MY PLAYLIST #55', 'spotify:playlist:6EY42J0jrEXLfqrdC95IDg'],
 ['the fall of jeff ...', 'spotify:playlist:6PaOb3LB9IDsA1lH5aHrYP'],
 ['SUMMER OF JEFF', 'spotify:playlist:5RZ4gZIM5lVg2blys9U1hy'],
 ['dilf training arc', 'spotify:playlist:3o2ad96K3umL8JJU8U049V'],
 ['im on that "from sklearn.preprocessing import MinMaxScaler" type of shit',
  'spotify:playlist:4s91FBXyfhCHpXpsoAbeP

In [14]:
test_playlist = user_playlists[12] #summer of jeff
test = sp.playlist(test_playlist[1])

In [15]:
#for parsing playlists, where test is a playlist's dict as directly retrieved by sp.playlist(ID)
test['uri'] #get uri
test['description']
test['images']
test['images'][0]['url'] #get direct url to cover image
test['name']
test['tracks']['total'] #number of tracks

test['tracks']['items'] #list of track info dicts

print(test['name'], ' has ', test['tracks']['total'], ' tracks')

SUMMER OF JEFF  has  92  tracks


In [16]:
#for parsing songs
song = test['tracks']['items'][0]
song['added_at']
song['track'].keys() #actual info dict
# includes ['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri']
song['track']['album']['name'] 
song['track']['album']['uri']
song['track']['album']['release_date']
song['track']['name']
song['track']['uri']
song['track']['popularity'] #0-100, based on internal algorithm, measurement of number and recency of streams
song['track']['artists'][0]['name'] #name of the top listed artist for a song, can also call 'uri'
song['track']['duration_ms']/1000 #duration of song in seconds
song['track']

features = sp.audio_features(song['track']['uri'])[0] #get audio features, including 'danceability' 'energy' 'key' 'loudness' 'mode'
features['danceability'] #0-1, measure of songs beat strength, regularity, rhythm, and tempo 
features['energy'] #0-1, measure of the perceived intensity of a song, including its 'loudness', predictability, and tempo
features['loudness'] #average dB loudness of the song, typically falling from 0-60
features['key'] #key the song is in from 0-11, or -1 if none detected, higher value = higher average pitch of song
features['mode'] #modality, 0 = minor, 1 = major
features['speechiness'] #0-1, spoken-wordness of the track, 1 means no music, most songs are below 0.33
features['acousticness'] #0-1, 1 = more acoustic, 0 = more electronic
features['instrumentalness'] #0-1, 1 = more instrumental and less speech, 0 = vice versa
features['liveness'] #0-1, higher value means more likelihood the song is recorded live
features['tempo'] #average/estimated BPM of song
features['valence'] #0-1, musical 'positiveness' of track with 1 being more uplifting and 0 being more downbeat
features['time_signature'] #3-7, where x means the song has a x/4 signature

song['track']['album']['release_date']

'2000'

In [34]:
test_playlist = user_playlists[12] #summer of jeff
def playlist_df(playlist:str):
    # given an input string (either a playlist's URI, URL, or ID), return a dataframe with info for each track in the playlist
    test = sp.playlist(playlist)
    tracks = []
    for song in test['tracks']['items']:
        features = sp.audio_features(song['track']['uri'])[0]
        tracks.append({'name':song['track']['name'],
                       'uri':song['track']['uri'],
                       'artist':song['track']['artists'][0]['name'],
                       'album':song['track']['album']['name'],
                       'duration_s':(song['track']['duration_ms']/1000),
                       'release':song['track']['album']['release_date'],   
                       'danceability':features['danceability'],
                       'energy':features['energy'],
                       'loudness':features['loudness'],
                       'speechiness':features['speechiness'],
                       'instrumentalness':features['instrumentalness'],
                       'acousticness':features['acousticness'],
                       'liveness':features['liveness'],
                       'tempo':features['tempo'],
                       'mode':features['mode'],
                       'key':features['key'],
                       'time_signature':features['time_signature'],
                      })
    return pd.DataFrame.from_records(tracks)

df = playlist_df('https://open.spotify.com/playlist/5bm2KdJ3GVF1IiDOK36wiQ?si=71303f14f2b34c94')

In [30]:
def get_song_details(song:str): 
    #given an input string (a song's URI, URL, or ID), return a dict with relevant information
    features = sp.audio_features(song)[0]
    return ({'name':song['track']['name'],
               'uri':song['track']['uri'],
               'artist':song['track']['artists'][0]['name'],
               'album':song['track']['album']['name'],
               'duration_s':(song['track']['duration_ms']/1000),
               'release':song['track']['album']['release_date'],   
               'danceability':features['danceability'],
               'energy':features['energy'],
               'loudness':features['loudness'],
               'speechiness':features['speechiness'],
               'instrumentalness':features['instrumentalness'],
               'acousticness':features['acousticness'],
               'liveness':features['liveness'],
               'tempo':features['tempo'],
               'mode':features['mode'],
               'key':features['key'],
               'time_signature':features['time_signature'],
              })

In [29]:
history = sp.current_user_recently_played(limit=10)
history['items'][0]

{'track': {'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5hAhrnb0Ch4ODwWu4tsbpi'},
     'href': 'https://api.spotify.com/v1/artists/5hAhrnb0Ch4ODwWu4tsbpi',
     'id': '5hAhrnb0Ch4ODwWu4tsbpi',
     'name': 'Yo La Tengo',
     'type': 'artist',
     'uri': 'spotify:artist:5hAhrnb0Ch4ODwWu4tsbpi'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    

In [10]:
def get_spotify_track_details(track_name, artist_name):
    #client_credentials_manager = SpotifyClientCredentials(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET')
    #sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    # Search for the input song
    search_results = sp.search(q=f'track:{track_name} artist:{artist_name}', type='track', limit=1)
    if not search_results['tracks']['items']:
        return None
    track = search_results['tracks']['items'][0]
    return track

def get_similar_tracks(track_id, limit=10):
    #client_credentials_manager = SpotifyClientCredentials(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET')
    #sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # Get audio features of the input track
    audio_features = sp.audio_features(track_id)
    if not audio_features:
        return None

    # Extract relevant audio features for similarity comparison
    input_features = audio_features[0]
    target_features = [input_features['danceability'],
                       input_features['energy'],
                       input_features['speechiness'],
                       input_features['acousticness'],
                       input_features['instrumentalness'],
                       input_features['liveness'],
                       input_features['valence'],
                       input_features['tempo']]

    # Get recommendations based on audio features
    recommended_tracks = sp.recommendations(seed_tracks=[track_id], limit=limit, target_audio_features=target_features)

    return recommended_tracks['tracks']

if __name__ == "__main__":
    input_song_name = 'i want wind to blow'
    input_artist_name = 'black countryoad'

    track_details = get_spotify_track_details(input_song_name, input_artist_name)

    if track_details:
        input_track_id = track_details['id']
        similar_tracks = get_similar_tracks(input_track_id, limit=10)

        print(f"Top 10 songs similar to '{input_song_name}' by {input_artist_name}:")
        for idx, track in enumerate(similar_tracks, start=1):
            print(f"{idx}. {track['name']} - {', '.join([artist['name'] for artist in track['artists']])}")
    else:
        print(f"No matching track found for '{input_song_name}' by {input_artist_name}.")


Top 10 songs similar to 'chaos space marine' by black country new road:
1. Spiritual Level Of Gang Shit - Soul Glo, McKinley Dixon, lojii
2. End of the Empire I-III - Arcade Fire
3. Bin Liner Fashion - Gilla Band
4. neon glow - glass beach
5. I Did Something Weird Last Night - Jeff Rosenstock
6. Everyone's Tongue - Horsey
7. Funeral Solution - Thee Oh Sees, Osees
8. When Life Hands You Problems - Cheekface
9. - - Porridge Radio
10. The Frontier Index - Silver Jews


In [16]:

lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

results = sp.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

track    : Immigrant Song - Remaster
audio    : https://p.scdn.co/mp3-preview/f50f3416ea0807503e172ba87a92d8047d4f95da?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b27390a50cfe99a4c19ff3cbfbdb

track    : Stairway to Heaven - Remaster
audio    : https://p.scdn.co/mp3-preview/337c8d45ac66899bc2db9ef3ed9438fe1d035ab1?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track    : Whole Lotta Love - 1990 Remaster
audio    : https://p.scdn.co/mp3-preview/89f3dc45265602fa874c23d638538af0f2c6b56a?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273fc4f17340773c6c3579fea0d

track    : Black Dog - Remaster
audio    : https://p.scdn.co/mp3-preview/78dd352fdbe8d2a867b72c5a71ac44ef6a462ab4?cid=c5029ed3aca649949ef00903e6aabf91
cover art: https://i.scdn.co/image/ab67616d0000b273c8a11e48c91a982d086afc69

track    : Rock and Roll - Remaster
audio    : https://p.scdn.